# Search Engine

## NLP: Document Similarity

### Keys Concepts
- Term Document Matrix
- Cosine Similarity

In [1]:
import warnings
from collections import OrderedDict
from pathlib import Path
from random import randint
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

# sklearn for feature extraction & modeling
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
# from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import train_test_split
# from sklearn.externals import joblib
import joblib

# Iteratively read files
import glob
import os

# For displaying images in ipython
from IPython.display import HTML, display

In [2]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14.0, 8.7)
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

### Load data

In [5]:
# User defined function to read and store bbc data from multipe folders
def load_data(folder_names,root_path):
    fileNames = [path + '/' + 'bbc' +'/'+ folder + '/*.txt' for path,folder in zip([root_path]*len(folder_names),
                                                                               folder_names )]
    doc_list = []
    tags = folder_names
    for docs in fileNames:
        print(docs)
        #print(type(docs))
        doc = glob.glob(docs) # glob method iterates through all the text documents in a folder
        for text in doc:
            with open(text, encoding='latin1') as f:
                topic = docs.split('/')[8]

                lines = f.readlines()
                heading = lines[0].strip()
                body = ' '.join([l.strip() for l in lines[1:]])
                doc_list.append([topic, heading, body])
        print("Completed loading data from folder: %s"%topic)
    
    print("Completed Loading entire text")
    
    return doc_list

In [4]:
os.getcwd().split()

['/Users/paragpradhan/Documents/Data',
 'Science',
 'Course/DSB6/5.',
 'Text',
 'Mining']

In [6]:
folder_names = ['business','entertainment','politics','sport','tech']
docs = load_data(folder_names = folder_names, root_path = os.getcwd())

/Users/paragpradhan/Documents/Data Science Course/DSB6/5. Text Mining/bbc/business/*.txt
Completed loading data from folder: business
/Users/paragpradhan/Documents/Data Science Course/DSB6/5. Text Mining/bbc/entertainment/*.txt
Completed loading data from folder: entertainment
/Users/paragpradhan/Documents/Data Science Course/DSB6/5. Text Mining/bbc/politics/*.txt
Completed loading data from folder: politics
/Users/paragpradhan/Documents/Data Science Course/DSB6/5. Text Mining/bbc/sport/*.txt
Completed loading data from folder: sport
/Users/paragpradhan/Documents/Data Science Course/DSB6/5. Text Mining/bbc/tech/*.txt
Completed loading data from folder: tech
Completed Loading entire text


In [7]:
docs = pd.DataFrame(docs, columns=['Category', 'Heading', 'Article'])
print(docs.head())
print('\nShape of data is {}\n'.format(docs.shape))
# print(docs.info())

   Category                            Heading  \
0  business    UK economy facing 'major risks'   
1  business  Aids and climate top Davos agenda   
2  business   Asian quake hits European shares   
3  business   India power shares jump on debut   
4  business    Lacroix label bought by US firm   

                                             Article  
0   The UK manufacturing sector will continue to ...  
1   Climate change and the fight against Aids are...  
2   Shares in Europe's leading reinsurers and tra...  
3   Shares in India's largest power producer, Nat...  
4   Luxury goods group LVMH has sold its loss-mak...  

Shape of data is (2225, 3)



<h2>Documents Similarity</h2>
<h3>From Documents -- DTM -- Cosine Similarity</h3>

HTML("<table><tr><td><img src="images/docs_to_dtm.png" alt="dtm" style="width:100%"></td><td><img src="images/cosine.jpg" alt="Forest" style="width:100%"></td></tr></table>")

<br>


### Important
The cosine similarity is the cosine of the angle between two vectors.
- Cosine Similarity can take value between 0 to 1.
- closer to 0 means dissimilar documents
- closer to 1 means similar documents

## Find Documents Similar to a New document : First Step to Build Search Engine

### Convert Raw text --> Parsed Text --> Document Term Matrix

### Why to Use Term Frequency Inverse Document Frequency over Term Frequency

- TF*IDF is an information retrieval technique that weighs a term’s frequency (TF) and its inverse document frequency (IDF). Each word or term has its respective TF and IDF score. The product of the TF and IDF scores of a term is called the TF*IDF weight of that term.

__Put simply, the higher the TF*IDF score (weight), the rarer the term and vice versa.__

__TFidf__ - is comprised of following two components
- __TF: Term Frequency__, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: 

__TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).__

__IDF: Inverse Document Frequency__, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: 

__IDF(t) = log_e(Total number of documents / Number of documents with term t in it).__

__TFidf__ = TF * IDF

**Important**
Higher the TFidf , more important the significance of word to that document

### Important Reading Article on TFidf
https://www.kdnuggets.com/2018/08/wtf-tf-idf.html

In [8]:
vectorizer = TfidfVectorizer(stop_words = "english")

In [9]:
vectors = vectorizer.fit_transform(docs["Heading"].values)
print("Shape of tfidf matrix: {}".format(vectors.shape))

Shape of tfidf matrix: (2225, 3623)


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
new_query = ["World facing imminent danger across global war theaters"]
new_query_vector = vectorizer.transform(new_query)
new_query_vector

<1x3623 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [13]:
sim = cosine_similarity(X = vectors, Y = new_query_vector)

In [14]:
sim

array([[0.26626668],
       [0.        ],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [15]:
# Extract Index of Maximum valued similar document
argmax = np.argmax(sim)
print("Index of maximum valued similar doc: %s"%argmax)
print("Retrieved Document Header: %s"%docs["Heading"][argmax])

Index of maximum valued similar doc: 438
Retrieved Document Header: The 'ticking budget' facing the US


In [16]:
# To Extract Top 10 Similar Documents against the new query
ind = np.argsort(sim,axis = 0)[::-1][:10]
for i in ind:
    print(docs["Heading"].values[i])

["The 'ticking budget' facing the US"]
["Singer Ferguson 'facing eviction'"]
['Spector facing more legal action']
["UK economy facing 'major risks'"]
['Big war games battle it out']
['Global release for Japan hit film']
['Bollywood draws global stars']
['Films on war triumph at Sundance']
["Global digital divide 'narrowing'"]
['The gaming world in 2005']


In [17]:
def retrieve_doc(new_query,raw_docs):
    vectorizer = TfidfVectorizer(stop_words = "english")
    vectors = vectorizer.fit_transform(raw_docs["Article"])
    print("Shape of tfidf matrix: {}".format(vectors.shape))
    new_query = [new_query]
    new_query_vector = vectorizer.transform(new_query)
    sim = cosine_similarity(X = vectors, Y = new_query_vector)
    ind = np.argsort(sim,axis = 0)[::-1][:10]
    for i in ind:
        print(docs["Heading"].values[i])

In [24]:
# newQuery = "gaming week in UK"
newQuery = "england to play australia in next ashes series"

In [25]:
retrieve_doc(new_query= newQuery  , raw_docs= docs)

Shape of tfidf matrix: (2225, 28980)
['Robinson wants dual code success']
['Wilkinson to lead England']
['Barkley fit for match in Ireland']
['Yachvili savours France comeback']
['Dallaglio his own man to the end']
['X Factor show gets second series']
['A November to remember']
['England given tough Sevens draw']
['Dawson set for new Wasps contract']
["O'Gara revels in Ireland victory"]
